# 05 - Modelo Descriptivo: Clasificación de Riesgo Mensual

## Descripción del Dataset
Dataset agregado por **municipio–año–mes** que clasifica el nivel de riesgo delictivo (BAJO, MEDIO, ALTO) basado en el total de delitos y variables sociodemográficas.

## Objetivo del Notebook
Generar **estadísticas pre-calculadas** en formato JSON para uso inmediato en:
- **Tableros de visualización** (dashboards)
- **Chatbots** con respuestas instantáneas
- **APIs** de consulta rápida

## Archivos Generados
| Archivo | Contenido |
|---------|-----------|
| `estadisticas_generales.json` | Métricas globales del dataset |
| `riesgo_por_municipio.json` | Análisis detallado por municipio |
| `tendencias_temporales.json` | Patrones por año y mes (estacionalidad) |
| `analisis_delitos.json` | Estadísticas por tipo de delito |
| `respuestas_chatbot.json` | Respuestas pre-generadas para preguntas frecuentes |

## Preguntas que Responde
- ¿Cuál es el municipio más/menos seguro?
- ¿Qué mes tiene mayor riesgo delictivo?
- ¿Cuál es el delito más común?
- ¿Cómo ha evolucionado el riesgo por año?
- ¿Qué tendencia tiene cada municipio?

## Utilidad Práctica
- **Respuestas instantáneas** sin necesidad de procesar datos en tiempo real
- **Priorización de recursos** según nivel de riesgo por municipio
- **Identificación de patrones estacionales** para prevención

In [1]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from datetime import datetime

# Rutas
BASE_DIR = Path().resolve().parent
DATA_PATH = BASE_DIR / 'data' / 'gold' / 'model' / 'classification_monthly_dataset.parquet'
OUTPUT_DIR = BASE_DIR / 'models' / 'descriptivo' / 'classification_monthly'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 Datos: {DATA_PATH}")
print(f"📁 Salida: {OUTPUT_DIR}")

📁 Datos: /Users/byverbel/quick_projects/Datos-al-Ecosistema/data/gold/model/classification_monthly_dataset.parquet
📁 Salida: /Users/byverbel/quick_projects/Datos-al-Ecosistema/models/descriptivo/classification_monthly


In [2]:
# Cargar datos
df = pd.read_parquet(DATA_PATH)
print(f" Dataset cargado: {df.shape[0]:,} registros, {df.shape[1]} columnas")
print(f" Período: {df['anio'].min()} - {df['anio'].max()}")
print(f" Municipios: {df['codigo_municipio'].nunique()}")

 Dataset cargado: 13,044 registros, 62 columnas
 Período: 2010 - 2025
 Municipios: 87


---
## 1. Estadísticas Generales

In [3]:
# 1.1 Calcular estadísticas generales
print(" CALCULANDO ESTADÍSTICAS GENERALES")
print("=" * 60)

estadisticas_generales = {
    'fecha_generacion': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'dataset': 'classification_monthly_dataset.parquet',
    'periodo': {
        'inicio': int(df['anio'].min()),
        'fin': int(df['anio'].max()),
        'total_meses': len(df['anio'].unique()) * 12
    },
    'cobertura': {
        'total_registros': int(len(df)),
        'municipios': int(df['codigo_municipio'].nunique()),
        'registros_por_municipio': round(len(df) / df['codigo_municipio'].nunique(), 1)
    },
    'distribucion_riesgo': {
        nivel: {
            'cantidad': int(count),
            'porcentaje': round(count / len(df) * 100, 2)
        }
        for nivel, count in df['nivel_riesgo'].value_counts().items()
    },
    'delitos': {
        'total_acumulado': int(df['total_delitos'].sum()),
        'promedio_mensual': round(df['total_delitos'].mean(), 2),
        'maximo_mensual': int(df['total_delitos'].max()),
        'municipio_max_delitos': int(df.loc[df['total_delitos'].idxmax(), 'codigo_municipio'])
    }
}

print(json.dumps(estadisticas_generales, indent=2, ensure_ascii=False))

 CALCULANDO ESTADÍSTICAS GENERALES
{
  "fecha_generacion": "2025-11-28 22:10:05",
  "dataset": "classification_monthly_dataset.parquet",
  "periodo": {
    "inicio": 2010,
    "fin": 2025,
    "total_meses": 192
  },
  "cobertura": {
    "total_registros": 13044,
    "municipios": 87,
    "registros_por_municipio": 149.9
  },
  "distribucion_riesgo": {
    "MEDIO": {
      "cantidad": 4563,
      "porcentaje": 34.98
    },
    "BAJO": {
      "cantidad": 4359,
      "porcentaje": 33.42
    },
    "ALTO": {
      "cantidad": 4122,
      "porcentaje": 31.6
    }
  },
  "delitos": {
    "total_acumulado": 508227,
    "promedio_mensual": 38.96,
    "maximo_mensual": 1723,
    "municipio_max_delitos": 68001
  }
}


---
## 2. Análisis por Municipio

In [4]:
# 2.1 Estadísticas por municipio
print(" CALCULANDO ESTADÍSTICAS POR MUNICIPIO")
print("=" * 60)

riesgo_por_municipio = {}

for cod_mun in df['codigo_municipio'].unique():
    df_mun = df[df['codigo_municipio'] == cod_mun]
    
    # Distribución de riesgo
    dist_riesgo = df_mun['nivel_riesgo'].value_counts(normalize=True) * 100
    
    # Nivel predominante
    nivel_predominante = df_mun['nivel_riesgo'].mode().iloc[0] if len(df_mun) > 0 else 'N/A'
    
    # Tendencia (comparar último año vs anterior)
    ultimo_anio = df_mun['anio'].max()
    delitos_ultimo = df_mun[df_mun['anio'] == ultimo_anio]['total_delitos'].sum()
    delitos_anterior = df_mun[df_mun['anio'] == ultimo_anio - 1]['total_delitos'].sum()
    
    if delitos_anterior > 0:
        tendencia_pct = round((delitos_ultimo - delitos_anterior) / delitos_anterior * 100, 2)
    else:
        tendencia_pct = 0
    
    riesgo_por_municipio[int(cod_mun)] = {
        'total_registros': int(len(df_mun)),
        'nivel_predominante': nivel_predominante,
        'distribucion_riesgo': {
            'BAJO': round(dist_riesgo.get('BAJO', 0), 2),
            'MEDIO': round(dist_riesgo.get('MEDIO', 0), 2),
            'ALTO': round(dist_riesgo.get('ALTO', 0), 2)
        },
        'delitos': {
            'total': int(df_mun['total_delitos'].sum()),
            'promedio_mensual': round(df_mun['total_delitos'].mean(), 2),
            'maximo_mensual': int(df_mun['total_delitos'].max())
        },
        'tendencia': {
            'variacion_anual_pct': tendencia_pct,
            'direccion': 'incremento' if tendencia_pct > 5 else ('decremento' if tendencia_pct < -5 else 'estable')
        },
        'poblacion_total': int(df_mun['poblacion_total'].iloc[-1]) if 'poblacion_total' in df_mun.columns else 0,
        'densidad_poblacional': round(df_mun['densidad_poblacional'].iloc[-1], 2) if 'densidad_poblacional' in df_mun.columns else 0
    }

print(f" Procesados {len(riesgo_por_municipio)} municipios")

# Mostrar top 5 municipios con más riesgo alto
print("\n Top 5 municipios con mayor % de riesgo ALTO:")
sorted_mun = sorted(riesgo_por_municipio.items(), 
                    key=lambda x: x[1]['distribucion_riesgo']['ALTO'], 
                    reverse=True)[:5]
for cod, data in sorted_mun:
    print(f"   {cod}: {data['distribucion_riesgo']['ALTO']:.1f}% ALTO")

 CALCULANDO ESTADÍSTICAS POR MUNICIPIO
 Procesados 87 municipios

 Top 5 municipios con mayor % de riesgo ALTO:
   68001: 100.0% ALTO
   68077: 100.0% ALTO
   68081: 100.0% ALTO
   68190: 100.0% ALTO
   68276: 100.0% ALTO


---
## 3. Tendencias Temporales

In [5]:
# 3.1 Tendencias por año y mes
print(" CALCULANDO TENDENCIAS TEMPORALES")
print("=" * 60)

tendencias_temporales = {
    'por_anio': {},
    'por_mes': {},
    'estacionalidad': {}
}

# Por año
for anio in sorted(df['anio'].unique()):
    df_anio = df[df['anio'] == anio]
    dist = df_anio['nivel_riesgo'].value_counts(normalize=True) * 100
    
    tendencias_temporales['por_anio'][int(anio)] = {
        'total_delitos': int(df_anio['total_delitos'].sum()),
        'promedio_mensual': round(df_anio['total_delitos'].mean(), 2),
        'distribucion_riesgo': {
            'BAJO': round(dist.get('BAJO', 0), 2),
            'MEDIO': round(dist.get('MEDIO', 0), 2),
            'ALTO': round(dist.get('ALTO', 0), 2)
        }
    }

# Por mes (estacionalidad)
meses_nombres = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
                 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in range(1, 13):
    df_mes = df[df['mes'] == mes]
    dist = df_mes['nivel_riesgo'].value_counts(normalize=True) * 100
    
    tendencias_temporales['por_mes'][mes] = {
        'nombre': meses_nombres[mes-1],
        'total_delitos': int(df_mes['total_delitos'].sum()),
        'promedio': round(df_mes['total_delitos'].mean(), 2),
        'pct_riesgo_alto': round(dist.get('ALTO', 0), 2)
    }

# Identificar meses más peligrosos
meses_ordenados = sorted(tendencias_temporales['por_mes'].items(),
                         key=lambda x: x[1]['pct_riesgo_alto'],
                         reverse=True)

tendencias_temporales['estacionalidad'] = {
    'meses_mas_riesgo': [{'mes': m, 'nombre': d['nombre'], 'pct_alto': d['pct_riesgo_alto']} 
                         for m, d in meses_ordenados[:3]],
    'meses_menos_riesgo': [{'mes': m, 'nombre': d['nombre'], 'pct_alto': d['pct_riesgo_alto']} 
                           for m, d in meses_ordenados[-3:]]
}

print("\n Meses con MAYOR riesgo:")
for item in tendencias_temporales['estacionalidad']['meses_mas_riesgo']:
    print(f"   {item['nombre']}: {item['pct_alto']:.1f}% riesgo alto")

print("\n Meses con MENOR riesgo:")
for item in tendencias_temporales['estacionalidad']['meses_menos_riesgo']:
    print(f"   {item['nombre']}: {item['pct_alto']:.1f}% riesgo alto")

 CALCULANDO TENDENCIAS TEMPORALES

 Meses con MAYOR riesgo:
   Enero: 33.8% riesgo alto
   Marzo: 32.6% riesgo alto
   Julio: 32.3% riesgo alto

 Meses con MENOR riesgo:
   Octubre: 31.0% riesgo alto
   Noviembre: 30.4% riesgo alto
   Febrero: 30.2% riesgo alto


---
## 4. Análisis por Tipo de Delito

In [6]:
# 4.1 Estadísticas por tipo de delito
print(" CALCULANDO ESTADÍSTICAS POR TIPO DE DELITO")
print("=" * 60)

tipos_delito = ['ABIGEATO', 'AMENAZAS', 'DELITOS SEXUALES', 'EXTORSION',
                'HOMICIDIOS', 'HURTOS', 'LESIONES', 'VIOLENCIA INTRAFAMILIAR']

# Filtrar solo los que existen
tipos_delito = [t for t in tipos_delito if t in df.columns]

analisis_delitos = {}

for tipo in tipos_delito:
    total = df[tipo].sum()
    promedio = df[tipo].mean()
    
    # Municipio con más casos
    casos_por_mun = df.groupby('codigo_municipio')[tipo].sum()
    mun_max = casos_por_mun.idxmax()
    
    # Tendencia anual
    por_anio = df.groupby('anio')[tipo].sum()
    if len(por_anio) >= 2:
        tendencia = (por_anio.iloc[-1] - por_anio.iloc[-2]) / por_anio.iloc[-2] * 100 if por_anio.iloc[-2] > 0 else 0
    else:
        tendencia = 0
    
    analisis_delitos[tipo] = {
        'total_historico': int(total),
        'promedio_mensual': round(promedio, 2),
        'porcentaje_del_total': round(total / df['total_delitos'].sum() * 100, 2),
        'municipio_mas_casos': int(mun_max),
        'tendencia_anual_pct': round(tendencia, 2)
    }

# Ordenar por total
delitos_ordenados = sorted(analisis_delitos.items(), 
                           key=lambda x: x[1]['total_historico'], 
                           reverse=True)

print("\n Ranking de delitos por frecuencia:")
for i, (tipo, data) in enumerate(delitos_ordenados, 1):
    print(f"   {i}. {tipo}: {data['total_historico']:,} ({data['porcentaje_del_total']:.1f}%)")

 CALCULANDO ESTADÍSTICAS POR TIPO DE DELITO

 Ranking de delitos por frecuencia:
   1. HURTOS: 172,321 (33.9%)
   2. LESIONES: 167,682 (33.0%)
   3. VIOLENCIA INTRAFAMILIAR: 89,619 (17.6%)
   4. AMENAZAS: 39,814 (7.8%)
   5. DELITOS SEXUALES: 24,594 (4.8%)
   6. HOMICIDIOS: 10,122 (2.0%)
   7. EXTORSION: 2,728 (0.5%)
   8. ABIGEATO: 1,347 (0.3%)


---
## 5. Generar Respuestas para Chatbot

In [7]:
# 5.1 Crear respuestas pre-generadas para el chatbot
print(" GENERANDO RESPUESTAS PARA CHATBOT")
print("=" * 60)

# Top municipios más seguros y menos seguros
mun_mas_seguros = sorted(riesgo_por_municipio.items(),
                         key=lambda x: x[1]['distribucion_riesgo']['BAJO'],
                         reverse=True)[:5]

mun_menos_seguros = sorted(riesgo_por_municipio.items(),
                           key=lambda x: x[1]['distribucion_riesgo']['ALTO'],
                           reverse=True)[:5]

respuestas_chatbot = {
    'municipios_mas_seguros': [
        {
            'codigo': cod,
            'pct_bajo': data['distribucion_riesgo']['BAJO'],
            'promedio_delitos': data['delitos']['promedio_mensual']
        }
        for cod, data in mun_mas_seguros
    ],
    'municipios_menos_seguros': [
        {
            'codigo': cod,
            'pct_alto': data['distribucion_riesgo']['ALTO'],
            'promedio_delitos': data['delitos']['promedio_mensual']
        }
        for cod, data in mun_menos_seguros
    ],
    'meses_mas_peligrosos': tendencias_temporales['estacionalidad']['meses_mas_riesgo'],
    'meses_mas_seguros': tendencias_temporales['estacionalidad']['meses_menos_riesgo'],
    'delito_mas_frecuente': {
        'tipo': delitos_ordenados[0][0],
        'total': delitos_ordenados[0][1]['total_historico'],
        'porcentaje': delitos_ordenados[0][1]['porcentaje_del_total']
    },
    'resumen_general': {
        'total_delitos': estadisticas_generales['delitos']['total_acumulado'],
        'promedio_mensual_departamento': estadisticas_generales['delitos']['promedio_mensual'],
        'municipios_analizados': estadisticas_generales['cobertura']['municipios'],
        'periodo': f"{estadisticas_generales['periodo']['inicio']}-{estadisticas_generales['periodo']['fin']}"
    },
    'preguntas_frecuentes': {
        '¿Cuál es el municipio más seguro?': f"El municipio con código {mun_mas_seguros[0][0]} tiene {mun_mas_seguros[0][1]['distribucion_riesgo']['BAJO']:.1f}% de meses en riesgo BAJO.",
        '¿Cuál es el mes más peligroso?': f"{tendencias_temporales['estacionalidad']['meses_mas_riesgo'][0]['nombre']} tiene {tendencias_temporales['estacionalidad']['meses_mas_riesgo'][0]['pct_alto']:.1f}% de riesgo alto.",
        '¿Qué delito es más común?': f"{delitos_ordenados[0][0]} representa el {delitos_ordenados[0][1]['porcentaje_del_total']:.1f}% del total de delitos.",
        '¿Cuántos delitos hay en promedio?': f"En promedio hay {estadisticas_generales['delitos']['promedio_mensual']:.1f} delitos por municipio al mes."
    }
}

print("\n Preguntas frecuentes generadas:")
for pregunta, respuesta in respuestas_chatbot['preguntas_frecuentes'].items():
    print(f"\n   Q: {pregunta}")
    print(f"   A: {respuesta}")

 GENERANDO RESPUESTAS PARA CHATBOT

 Preguntas frecuentes generadas:

   Q: ¿Cuál es el municipio más seguro?
   A: El municipio con código 68686 tiene 92.3% de meses en riesgo BAJO.

   Q: ¿Cuál es el mes más peligroso?
   A: Enero tiene 33.8% de riesgo alto.

   Q: ¿Qué delito es más común?
   A: HURTOS representa el 33.9% del total de delitos.

   Q: ¿Cuántos delitos hay en promedio?
   A: En promedio hay 39.0 delitos por municipio al mes.


---
## 6. Guardar Archivos JSON

In [8]:
# 6.1 Guardar todos los archivos JSON
print(" GUARDANDO ARCHIVOS JSON")
print("=" * 60)

archivos = {
    'estadisticas_generales.json': estadisticas_generales,
    'riesgo_por_municipio.json': riesgo_por_municipio,
    'tendencias_temporales.json': tendencias_temporales,
    'analisis_delitos.json': analisis_delitos,
    'respuestas_chatbot.json': respuestas_chatbot
}

for nombre, datos in archivos.items():
    path = OUTPUT_DIR / nombre
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(datos, f, indent=2, ensure_ascii=False)
    print(f" {nombre}")

print(f"\n Archivos guardados en: {OUTPUT_DIR}")

 GUARDANDO ARCHIVOS JSON
 estadisticas_generales.json
 riesgo_por_municipio.json
 tendencias_temporales.json
 analisis_delitos.json
 respuestas_chatbot.json

 Archivos guardados en: /Users/byverbel/quick_projects/Datos-al-Ecosistema/models/descriptivo/classification_monthly


---
## 7. Ejemplo de Uso

In [9]:
# 7.1 Ejemplo de cómo usar los datos en el tablero/chatbot
print(" EJEMPLO DE USO EN TABLERO/CHATBOT")
print("=" * 60)

ejemplo = '''
# En el backend del tablero o chatbot:

import json

# Cargar respuestas pre-generadas
with open('models/descriptivo/classification_monthly/respuestas_chatbot.json', 'r') as f:
    respuestas = json.load(f)

# Responder preguntas instantáneamente
def responder_pregunta(pregunta):
    if 'más seguro' in pregunta.lower():
        mun = respuestas['municipios_mas_seguros'][0]
        return f"El municipio {mun['codigo']} es el más seguro con {mun['pct_bajo']:.1f}% de riesgo bajo."
    
    if 'mes' in pregunta.lower() and 'peligroso' in pregunta.lower():
        mes = respuestas['meses_mas_peligrosos'][0]
        return f"{mes['nombre']} es el mes más peligroso con {mes['pct_alto']:.1f}% de riesgo alto."
    
    if 'delito' in pregunta.lower() and 'común' in pregunta.lower():
        delito = respuestas['delito_mas_frecuente']
        return f"{delito['tipo']} es el más común ({delito['porcentaje']:.1f}% del total)."
    
    return "Lo siento, no tengo información sobre eso."

# Ejemplo de uso
print(responder_pregunta("¿Cuál es el municipio más seguro?"))
print(responder_pregunta("¿Qué mes es el más peligroso?"))
print(responder_pregunta("¿Cuál es el delito más común?"))
'''

print(ejemplo)

 EJEMPLO DE USO EN TABLERO/CHATBOT

# En el backend del tablero o chatbot:

import json

# Cargar respuestas pre-generadas
with open('models/descriptivo/classification_monthly/respuestas_chatbot.json', 'r') as f:
    respuestas = json.load(f)

# Responder preguntas instantáneamente
def responder_pregunta(pregunta):
    if 'más seguro' in pregunta.lower():
        mun = respuestas['municipios_mas_seguros'][0]
        return f"El municipio {mun['codigo']} es el más seguro con {mun['pct_bajo']:.1f}% de riesgo bajo."

    if 'mes' in pregunta.lower() and 'peligroso' in pregunta.lower():
        mes = respuestas['meses_mas_peligrosos'][0]
        return f"{mes['nombre']} es el mes más peligroso con {mes['pct_alto']:.1f}% de riesgo alto."

    if 'delito' in pregunta.lower() and 'común' in pregunta.lower():
        delito = respuestas['delito_mas_frecuente']
        return f"{delito['tipo']} es el más común ({delito['porcentaje']:.1f}% del total)."

    return "Lo siento, no tengo informació

In [10]:
# 7.2 Resumen final
print(" RESUMEN DEL MODELO DESCRIPTIVO")
print("=" * 60)

print(f"""
 ARCHIVOS GENERADOS:
   • estadisticas_generales.json - Métricas globales del dataset
   • riesgo_por_municipio.json - Análisis detallado por municipio
   • tendencias_temporales.json - Patrones por año y mes
   • analisis_delitos.json - Estadísticas por tipo de delito
   • respuestas_chatbot.json - Respuestas pre-generadas

 ESTADÍSTICAS CLAVE:
   • Período analizado: {estadisticas_generales['periodo']['inicio']}-{estadisticas_generales['periodo']['fin']}
   • Municipios: {estadisticas_generales['cobertura']['municipios']}
   • Total delitos: {estadisticas_generales['delitos']['total_acumulado']:,}
   • Promedio mensual: {estadisticas_generales['delitos']['promedio_mensual']:.1f}

 PREGUNTAS QUE PUEDE RESPONDER:
   • ¿Cuál es el municipio más/menos seguro?
   • ¿Qué mes tiene más riesgo?
   • ¿Cuál es el delito más común?
   • ¿Cómo evolucionó el riesgo por año?
   • ¿Qué tendencia tiene cada municipio?

 Modelo descriptivo completado exitosamente
 Ubicación: {OUTPUT_DIR}
""")

 RESUMEN DEL MODELO DESCRIPTIVO

 ARCHIVOS GENERADOS:
   • estadisticas_generales.json - Métricas globales del dataset
   • riesgo_por_municipio.json - Análisis detallado por municipio
   • tendencias_temporales.json - Patrones por año y mes
   • analisis_delitos.json - Estadísticas por tipo de delito
   • respuestas_chatbot.json - Respuestas pre-generadas

 ESTADÍSTICAS CLAVE:
   • Período analizado: 2010-2025
   • Municipios: 87
   • Total delitos: 508,227
   • Promedio mensual: 39.0

 PREGUNTAS QUE PUEDE RESPONDER:
   • ¿Cuál es el municipio más/menos seguro?
   • ¿Qué mes tiene más riesgo?
   • ¿Cuál es el delito más común?
   • ¿Cómo evolucionó el riesgo por año?
   • ¿Qué tendencia tiene cada municipio?

 Modelo descriptivo completado exitosamente
 Ubicación: /Users/byverbel/quick_projects/Datos-al-Ecosistema/models/descriptivo/classification_monthly

